In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.enrich_action import EnrichCEOrderCreatedJob
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-26 16:39:15.670 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-26 16:39:23.631 | INFO     | create_data:recreate_databases:29 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-26 16:39:23.682 | INFO     | create_data:recreate_databases:33 - Creating database 'standardized_glovo_live'
2021-11-26 16:39:26.365 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_custom_events'
2021-11-26 16:39:26.417 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_screen_views'
2021-11-26 16:39:26.465 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_custom_events'
2021-11-26 16:39:26.515 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-26 16:39:26.571 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.cities'
2021-11-26 16:39:35.643 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.devices'
2021-11-26 16:39:41.823 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.orders'
2021-11-26 16:39:48.283 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:39:54.305 | INFO     | create_data:create_table:91 - Creating table 'standardized_glovo_live.customer_subscription_periods'
2021-11-26 16:40:00.668 | INFO     | create_data:create_table:91 - Creating table 'mpcustomer_custom_events.order_created'


In [5]:
order_created_job = EnrichCEOrderCreatedJob(
    spark=spark,
    exec_date=date(2021, 11, 19),
    n_days=3
)
order_created_job.run()

2021-11-26 16:40:06.918 | INFO     | loaders:load:47 - Loading 'mpcustomer_custom_events.order_created'
2021-11-26 16:40:07.064 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.cities'
2021-11-26 16:40:07.171 | INFO     | loaders:load:47 - Loading 'standardized_glovo_live.orders'
2021-11-26 16:40:07.205 | INFO     | ports.live_db:load:72 - Loading 'standardized_glovo_live.customer_subscriptions'
2021-11-26 16:40:07.390 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-26 16:40:07.425 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddNumberOfOrders'
2021-11-26 16:40:07.783 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddIsPrime'
2021-11-26 16:40:07.920 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddLocalTime'
2021-11-26 16:40:08.071 | INFO     | writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [6]:
spark.table("mpcustomer_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|
+-----------------------+-----------+-------------------+---------------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|
|                    CAG|          2|2021-11-01 07:29:00|     2021-11-01|
+-----------------------+-----------+-------------------+---------------+



# Show outputs

In [7]:
spark.table("enriched_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+--------+-------------------+-----------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|    time_zone|country_code|number_of_orders|is_prime|creation_time_local|time_of_day|
+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+--------+-------------------+-----------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|Europe/Madrid|          ES|               2|   false|2021-11-19 13:00:00|      Lunch|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|Europe/Madrid|          ES|               2|   false|2021-11-18 09:00:00|  Breakfast|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|  Europe/Rome|          IT|               1|    true|2021-11-17 06:05:00|  Breakfast|
+-----------------------+-----------+---------